In [1]:
# import tensorflow as tf
import numpy as np
import h2o
import os
import pandas as pd

# from .feature_engineering_filter import Find_correct_port
import filters


In [2]:
train_data = pd.read_csv("../Datasets/ais_train.csv", delimiter="|")
test_data = pd.read_csv("../Datasets/ais_test.csv", delimiter=",")

In [3]:
train_data["time"] = pd.to_datetime(train_data["time"])
test_data["time"] = pd.to_datetime(test_data["time"])

In [4]:
train_data_preprocessed = train_data

train_data_preprocessed = filters.HandleMissingValues()(train_data_preprocessed)

train_data_preprocessed = train_data_preprocessed.sort_values("time")

print(train_data_preprocessed.head())

train_data_preprocessed = filters.GroupBy()(train_data_preprocessed, "vesselId")

train_data_preprocessed["heading"] = train_data_preprocessed["heading"].fillna(0)
train_data_preprocessed = train_data_preprocessed.dropna().reset_index(drop=True)

train_data_preprocessed = filters.CalculateSeconds()(train_data_preprocessed)

print(train_data_preprocessed.head())

                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-01 00:00:25  284.0   0.7  0.0     88.0        0  01-09 23:00   
1 2024-01-01 00:00:36  109.6   0.0 -6.0    347.0        1  12-29 20:00   
2 2024-01-01 00:01:45  111.0  11.0  0.0    112.0        0  01-02 09:00   
3 2024-01-01 00:03:11   96.4   0.0  0.0    142.0        1  12-31 20:00   
4 2024-01-01 00:03:51  214.0  19.7  0.0    215.0        0  01-25 12:00   

   latitude  longitude                  vesselId                    portId  
0 -34.74370  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f  
1   8.89440  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689  
2  39.19065  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19  
3 -34.41189  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126  
4  35.88379   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3  
                 time    cog   sog  rot  heading  navstat  latitude  \
0 2024-01-12 14:07:47 

In [5]:
train_data_preprocessed = filters.ConvertToTrigonometric()(train_data_preprocessed)
print(train_data_preprocessed.head(10))

                 time   sog  rot  navstat                  vesselId  \
0 2024-01-12 14:07:47  17.1 -6.0        0  61e9f38eb937134a3c4bfd8b   
1 2024-01-12 14:31:00  17.3  5.0        0  61e9f38eb937134a3c4bfd8b   
2 2024-01-12 14:57:23  16.9  5.0        0  61e9f38eb937134a3c4bfd8b   
3 2024-01-12 15:18:48  16.9  6.0        0  61e9f38eb937134a3c4bfd8b   
4 2024-01-12 15:39:47  16.3  7.0        0  61e9f38eb937134a3c4bfd8b   
5 2024-01-12 15:54:48  16.1  5.0        0  61e9f38eb937134a3c4bfd8b   
6 2024-01-12 16:14:59  16.1 -6.0        0  61e9f38eb937134a3c4bfd8b   
7 2024-01-12 16:35:24  16.0  2.0        0  61e9f38eb937134a3c4bfd8b   
8 2024-01-12 16:55:24  16.0 -1.0        0  61e9f38eb937134a3c4bfd8b   
9 2024-01-12 17:14:36  16.1  6.0        0  61e9f38eb937134a3c4bfd8b   

   seconds_to_eta  latitude_sin  latitude_cos  longitude_sin  longitude_cos  \
0       -374867.0      0.130589      0.991437       0.976610       0.215018   
1        205140.0      0.131790      0.991278       0.976277

In [6]:
print(train_data_preprocessed.columns)

train_data_shifted_df =filters.LastKnownLocation()(
    train_data_preprocessed, 400
)

# train_data_shifted_df = train_data_shifted_df.drop(columns=["time"], axis=1)

Index(['time', 'sog', 'rot', 'navstat', 'vesselId', 'seconds_to_eta',
       'latitude_sin', 'latitude_cos', 'longitude_sin', 'longitude_cos',
       'cog_sin', 'cog_cos', 'heading_sin', 'heading_cos', 'hour_sin',
       'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos'],
      dtype='object')
Shift length: 2
Shift length: 4
Shift length: 8


KeyboardInterrupt: 

In [ ]:
test_data_with_last_known_df = filters.AppendLastKnownData()(
    test_data, train_data_preprocessed
)
test_data_with_last_known_df[
    [
        "last_latitude_sin",
        "last_latitude_cos",
        "last_longitude_sin",
        "last_longitude_cos",
    ]
] = test_data_with_last_known_df[
    [
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ]
]
test_data_with_last_known_df = test_data_with_last_known_df.drop(
    columns=[
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ],
    axis=1,
)

print(test_data_with_last_known_df.head())

[]
   ID                  vesselId  scaling_factor                time   sog  \
0   0  61e9f3aeb937134a3c4bfe3d             0.3 2024-05-07 23:48:16   0.0   
1   1  61e9f473b937134a3c4c02df             0.3 2024-05-07 23:57:16   0.0   
2   2  61e9f469b937134a3c4c029b             0.3 2024-05-07 23:59:08  18.7   
3   3  61e9f45bb937134a3c4c0221             0.3 2024-05-07 23:52:34   0.1   
4   4  61e9f38eb937134a3c4bfd8d             0.3 2024-05-07 23:51:29   0.3   

   rot  navstat  seconds_to_eta   cog_sin   cog_cos  ...  hour_cos   day_sin  \
0  0.0        5       -133396.0  0.006981 -0.999976  ...  0.965926  0.994522   
1  0.0        5       -521836.0  0.417867  0.908508  ...  0.965926  0.994522   
2  0.0        0         45952.0  0.139173  0.990268  ...  0.965926  0.994522   
3  0.0        1        -81454.0 -0.625243  0.780430  ...  0.965926  0.994522   
4  0.0        2        101311.0 -0.933580  0.358368  ...  0.965926  0.994522   

    day_cos  month_sin  month_cos  time_diff  last_la

In [ ]:
h2o.init(max_mem_size="7g")

train_data_shifted = h2o.H2OFrame(train_data_shifted_df)
test_data_with_last_known = h2o.H2OFrame(test_data_with_last_known_df)

Checking whether there is an H2O instance running at http://localhost:54321....

In [ ]:
train_data_shifted_without_validation, validation_data_shifted = (
    train_data_shifted.split_frame(ratios=[0.8], seed=42)   
)               

In [ ]:
features_lat = [
    # "vesselId",
    "cog_sin",
    "cog_cos",
    "sog",
    # "rot",
    # "heading_sin",
    # "heading_cos",
    # "navstat",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    # "hour_sin",
    # "hour_cos",
    "day_sin",
    "day_cos",
    # "month_sin",
    # "month_cos",
]
features_long = [
    # "vesselId",
    "cog_sin",
    "cog_cos",
    "sog",
    # "rot",
    # "heading_sin",
    # "heading_cos",
    # "navstat",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    # "hour_sin",
    # "hour_cos",
    "day_sin",
    "day_cos",
    # "month_sin",
    # "month_cos",
    "latitude_sin",
    "latitude_cos",
]
target_long_sin = "longitude_sin"
target_long_cos = "longitude_cos"
target_lat_sin = "latitude_sin"
target_lat_cos = "latitude_cos"

In [ ]:
params = {
    "ntrees": 100,  # Maximum number of trees
    "max_depth": 8,  # Maximum depth of each tree
    "min_rows": 15,  # Minimum number of rows per leaf
    "learn_rate": 0.1,  # Learning rate
    "sample_rate": 0.8,  # Row sample rate per tree
    "col_sample_rate": 0.8,  # Column sample rate per tree
    "reg_lambda": 1.0,  # L2 regularization term
    "reg_alpha": 0.1,  # L1 regularization term
    "seed": 42,  # Random seed for reproducibility
    "stopping_rounds": 10,  # Early stopping rounds
    "stopping_metric": "AUTO",  # Metric for early stopping
    "stopping_tolerance": 0.001,  # Tolerance for early stopping
}
# score_eval_metric_only if you want to predict only on the evaluation dataset, could help against overfitting

gbm_lat_sin = h2o.estimators.H2OXGBoostEstimator(**params)
gbm_lat_cos = h2o.estimators.H2OXGBoostEstimator(**params)
gbm_long_sin = h2o.estimators.H2OXGBoostEstimator(**params)
gbm_long_cos = h2o.estimators.H2OXGBoostEstimator(**params)

# gbm_cog = h2o.estimators.H2OXGBoostEstimator()
# gbm_sog = h2o.estimators.H2OXGBoostEstimator()
# gbm_rot = h2o.estimators.H2OXGBoostEstimator()
# gbm_heading = h2o.estimators.H2OXGBoostEstimator()
# gbm_navstat = h2o.estimators.H2OXGBoostEstimator()
# # gbm_etaRaw = h2o.esti#mators.H2OXGBoostEstimator() #Remove etaRaw because it requires preprocessing
# # gbm_portId = h2o.estimators.H2OXGBoostEstimator()


In [ ]:
gbm_lat_sin.train(
    x=features_lat,
    y=target_lat_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

H2OResponseError: Server error water.exceptions.H2ONotFoundArgumentException:
  Error: POST /3/ModelBuilders/xgboost not found
  Request: POST /3/ModelBuilders/xgboost
    data: {'training_frame': 'py_2_sid_9a0a', 'validation_frame': 'py_3_sid_9a0a', 'nfolds': '0', 'keep_cross_validation_models': 'True', 'keep_cross_validation_predictions': 'False', 'keep_cross_validation_fold_assignment': 'False', 'score_each_iteration': 'False', 'fold_assignment': 'auto', 'response_column': 'latitude_sin', 'ignored_columns': '["longitude_sin","heading_cos","month_cos","hour_sin","hour_cos","rot","latitude_cos","time","navstat","vesselId","heading_sin","longitude_cos","month_sin"]', 'ignore_const_cols': 'True', 'stopping_rounds': '10', 'stopping_metric': 'AUTO', 'stopping_tolerance': '0.001', 'max_runtime_secs': '0.0', 'seed': '42', 'distribution': 'auto', 'tweedie_power': '1.5', 'categorical_encoding': 'auto', 'quiet_mode': 'True', 'ntrees': '100', 'max_depth': '8', 'min_rows': '15', 'min_child_weigh

In [ ]:
gbm_lat_cos.train(
    x=features_lat,  # .append(latitude_sin)
    y=target_lat_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729545096730_6


Model Summary: 
    number_of_trees
--  -----------------
    100

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.0009402392177612258
RMSE: 0.030663320396871988
MAE: 0.015588066940304525
RMSLE: 0.017171058810274302
Mean Residual Deviance: 0.0009402392177612258

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.0009673543463447165
RMSE: 0.03110232059420513
MAE: 0.015748499780178195
RMSLE: 0.017421605197001852
Mean Residual Deviance: 0.0009673543463447165

Scoring History: 
     timestamp            duration          number_of_trees    training_rmse         training_mae          training_deviance      validation_rmse       validation_mae        validation_deviance
---  -------------------  ----------------  -----------------  --------------------  --------------------  ---------------------  --------------------  --------------------  ---------------------
     2024-10-22 11:19:31  0.018 sec         0.0                0.26584536940232334   0.23343598017083866   0.07067376043265776    0.2656441633647033    0.23324266187658083   0.07056682152973319
     2024-10-22 11:19:40  8.817 sec         1.0                0.23994523555984917   0.21017883056751868   0.05757371606787151    0.23975857301156667   0.20999360569862288   0.05748417333254275
     2024-10-22 11:19:50  18.597 sec        3.0                0.19582159419310408   0.17055756397551028   0.03834609675232874    0.19566471664679416   0.1703922493174015    0.03828468134047025
     2024-10-22 11:19:59  28.070 sec        5.0                0.16033154994582274   0.13858246183938233   0.025706205908029852   0.16020653436203544   0.13844646624150528   0.025666133652294042
     2024-10-22 11:20:04  33.085 sec        6.0                0.14529919423866178   0.1250773918495538    0.021111855846404362   0.14519291091262296   0.12496291240174565   0.02108098137928087
     2024-10-22 11:20:13  42.381 sec        8.0                0.1198783959111937    0.10227017226310137   0.014370829806240904   0.11980333849820914   0.10219375505915103   0.01435283991531648
     2024-10-22 11:20:23  52.182 sec        10.0               0.09968124141348747   0.08407073664450745   0.00993634988973397    0.09962993539255295   0.08402127378596512   0.009926124026324274
     2024-10-22 11:20:33  1 min  2.049 sec  12.0               0.08371604445344515   0.06950815999751112   0.0070083760989312046  0.0836931880463632    0.06947852528208491   0.007004549725363913
     2024-10-22 11:20:43  1 min 12.092 sec  14.0               0.07127331073015171   0.057953788691149206  0.005079884822436758   0.07128144263336943   0.057944850299643795  0.005081044063894336
     2024-10-22 11:20:53  1 min 22.240 sec  16.0               0.06156298528162822   0.04880115686109592   0.003790001156785973   0.06160333597445897   0.048810736860718205  0.0037949710031820703
---  ---                  ---               ---                ---                   ---                   ---                    ---                   ---                   ---
     2024-10-22 11:22:28  2 min 56.626 sec  34.0               0.03512832944826329   0.019972813006316318  0.0012339995298257217  0.03536718181817035   0.020061283156762626  0.0012508375497595192
     2024-10-22 11:22:51  3 min 20.187 sec  38.0               0.0342113347602496    0.01852892631129797   0.0011704154260778628  0.03447589563969951   0.018627198534758006  0.0011885873801594515
     2024-10-22 11:23:17  3 min 46.081 sec  43.0               0.03346351427792395   0.01749030275577151   0.00111980678782882    0.03375008512293253   0.01759877761535826   0.0011390682458051918
     2024-10-22 11:23:43  4 min 12.268 sec  48.0               0.033050182088499476  0.016952259509989305  0.0010923145360829717  0.03334570648910659   0.017066453644532784  0.0011119361412576453
     2024-10-22 11:24:20  4 min 48.496 sec  55.0               0.0326

In [ ]:
performance_lat_sin = gbm_lat_sin.model_performance(test_data=validation_data_shifted)
performance_lat_cos = gbm_lat_cos.model_performance(test_data=validation_data_shifted)


# Print the performance metrics
print(performance_lat_sin)
print(performance_lat_cos)

ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.008285615274438071
RMSE: 0.09102535511843977
MAE: 0.03389776072872149
RMSLE: 0.09814127050811626
Mean Residual Deviance: 0.008285615274438071
ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.0009673543463447165
RMSE: 0.03110232059420513
MAE: 0.015748499780178195
RMSLE: 0.017421605197001852
Mean Residual Deviance: 0.0009673543463447165


In [ ]:
gbm_long_sin.train(
    x=features_long,
    y=target_long_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729545096730_7


Model Summary: 
    number_of_trees
--  -----------------
    100

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.0054584688377833195
RMSE: 0.07388145124307806
MAE: 0.02718451666557251
RMSLE: NaN
Mean Residual Deviance: 0.0054584688377833195

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.005702151680964747
RMSE: 0.07551259286347375
MAE: 0.02753314811772908
RMSLE: NaN
Mean Residual Deviance: 0.005702151680964747

Scoring History: 
     timestamp            duration           number_of_trees    training_rmse        training_mae          training_deviance      validation_rmse      validation_mae        validation_deviance
---  -------------------  -----------------  -----------------  -------------------  --------------------  ---------------------  -------------------  --------------------  ---------------------
     2024-10-22 11:28:34  0.022 sec          0.0                0.6973596157798081   0.5570897697431579    0.48631043372056154    0.6969687459619623   0.556833065740921     0.4857654328477904
     2024-10-22 11:28:44  10.310 sec         1.0                0.6306469442835649   0.5034928190730452    0.39771556833419786    0.6303497181858337   0.503285951771811     0.39734076721696
     2024-10-22 11:28:56  22.197 sec         3.0                0.5180660558047484   0.4125965953081795    0.26839243817708863    0.5179126527494827   0.4124701380820508    0.26823351587800626
     2024-10-22 11:29:08  34.705 sec         5.0                0.42755378061494226  0.33883288843715414   0.18280223531813017    0.4275328310842117   0.33877596803687066   0.1827843216548811
     2024-10-22 11:29:20  46.584 sec         7.0                0.3553770592930568   0.27926333564284656   0.1262928542717808     0.3554630655640049   0.2792415408029395    0.12635399098016006
     2024-10-22 11:29:32  58.249 sec         9.0                0.29762669512893847  0.23109787465687495   0.08858164965337408    0.297828751873222    0.23111126585052036   0.08870196544236122
     2024-10-22 11:29:44  1 min 10.090 sec   11.0               0.2521457826950387   0.19238194072490525   0.06357749573089366    0.25244615325295233  0.19242710552004083   0.06372906029221309
     2024-10-22 11:30:02  1 min 28.583 sec   14.0               0.20079396704008548  0.1478075152053829    0.04031821719969494    0.20120234374890536  0.14788100135395685   0.040482383130052674
     2024-10-22 11:30:14  1 min 40.220 sec   16.0               0.17576725546336203  0.1250615994419838    0.030894128093122766   0.17625710551415608  0.12514819848927072   0.03106656724422835
     2024-10-22 11:30:45  2 min 10.798 sec   21.0               0.13601574462011212  0.08621490871152301   0.018500282784563563   0.13664712900917675  0.08636633902810537   0.018672437866450592
---  ---                  ---                ---                ---                  ---                   ---                    ---                  ---                   ---
     2024-10-22 11:32:40  4 min  6.654 sec   38.0               0.09523358739797082  0.04024096214778666   0.009069436168686947   0.09614154654620673  0.04049938373585432   0.009243196972296435
     2024-10-22 11:33:17  4 min 43.583 sec   44.0               0.09023627225387933  0.0359251155335838    0.008142584830276233   0.09125572345080528  0.0361997198399074    0.008327607062529852
     2024-10-22 11:33:48  5 min 13.808 sec   49.0               0.08677493373082396  0.03363062193038337   0.007529889123988889   0.08790386710599422  0.03391847713823637   0.007727089852188293
     2024-10-22 11:34:17  5 min 43.298 sec   54.0               0.08485762514203651  0.03242347787940477   0.007200816544746388   0.08603816533646204  0.03271275401797562   0.007402565894464377
     2024-10-22 11:34:54  6 min 20.246 sec   60.0               0.08295178090423772  0.03121347896797209   0.006880997955184656   0.084205

In [ ]:
gbm_long_cos.train(
    x=features_long,  # .append("longitude_sin")
    y=target_long_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729545096730_8


Model Summary: 
    number_of_trees
--  -----------------
    100

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.0034551094601051684
RMSE: 0.0587801791431871
MAE: 0.01622333409483071
RMSLE: NaN
Mean Residual Deviance: 0.0034551094601051684

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.003737695274238994
RMSE: 0.0611366933538198
MAE: 0.016595873311879054
RMSLE: NaN
Mean Residual Deviance: 0.003737695274238994

Scoring History: 
     timestamp            duration           number_of_trees    training_rmse         training_mae          training_deviance      validation_rmse      validation_mae        validation_deviance
---  -------------------  -----------------  -----------------  --------------------  --------------------  ---------------------  -------------------  --------------------  ---------------------
     2024-10-22 11:39:04  0.016 sec          0.0                0.6768341919805017    0.6009565723345223    0.45810452343389857    0.6768841499777741   0.6010695364298099    0.4581721524911338
     2024-10-22 11:39:15  10.628 sec         1.0                0.6117255923666026    0.5429198642056088    0.3742082003562709     0.6117767362630273   0.543033164965465     0.3742707750326417
     2024-10-22 11:39:27  23.070 sec         3.0                0.5006075365022883    0.4436603980428929    0.25060790560288987    0.5006652639419976   0.4437833032059866    0.25066570651811015
     2024-10-22 11:39:46  41.591 sec         6.0                0.37300623775744074   0.3285428177960366    0.13913365340596043    0.37309604145439723  0.3286691004482215    0.1392006561489413
     2024-10-22 11:39:57  52.649 sec         8.0                0.3085640457307822    0.26943373377572216   0.09521177031774826    0.30870672417440986  0.26957273934868003   0.09529984155049517
     2024-10-22 11:40:09  1 min  4.296 sec   10.0               0.257188979885677     0.22171957441765755   0.06614617137463519    0.25739416685150746  0.22186484232167045   0.06625175712918166
     2024-10-22 11:40:20  1 min 15.622 sec   12.0               0.21628230751874739   0.18299154737248255   0.04677803654563401    0.216574491406325    0.1831474345480281    0.04690451032790834
     2024-10-22 11:40:31  1 min 27.245 sec   14.0               0.18401366940829844   0.15163361473157597   0.03386103052910655    0.18437774263825418  0.15178785768762784   0.033995151980378294
     2024-10-22 11:40:43  1 min 38.968 sec   16.0               0.15883750540779      0.12615762265707015   0.025229353124169712   0.1593074439748958   0.1263227093257482    0.025378861705814566
     2024-10-22 11:41:02  1 min 57.348 sec   19.0               0.13112234851144952   0.09679146284735816   0.017193070279158028   0.13175499150306078  0.09697182027230586   0.017359377785971616
---  ---                  ---                ---                ---                   ---                   ---                    ---                  ---                   ---
     2024-10-22 11:43:10  4 min  5.312 sec   40.0               0.07546250911791173   0.026207625492068277  0.0056945902823709125  0.07695476876756083  0.026477261460967193  0.005922036436068754
     2024-10-22 11:43:34  4 min 29.657 sec   44.0               0.0730487090408456    0.02340350265553394   0.005336113892534117   0.07463273577815654  0.02368189726281953   0.005570045249732128
     2024-10-22 11:44:03  4 min 59.058 sec   49.0               0.07082745462106779   0.021260345215325452  0.0050165283280994176  0.07245169776669763  0.02154017202898303   0.005249248509276899
     2024-10-22 11:44:34  5 min 29.600 sec   54.0               0.06893626807799143   0.01991800327621504   0.004752209056520701   0.07062414990328729  0.020207872368463786  0.004987770549561995
     2024-10-22 11:45:10  6 min  5.884 sec   60.0               0.06665519116048098   0.018898529457958935  0.004442914

In [ ]:
performance_long_sin = gbm_long_sin.model_performance(test_data=validation_data_shifted)
performance_long_cos = gbm_long_cos.model_performance(test_data=validation_data_shifted)

print(performance_long_sin)
print(performance_long_cos)

ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.005702151680964747
RMSE: 0.07551259286347375
MAE: 0.02753314811772908
RMSLE: NaN
Mean Residual Deviance: 0.005702151680964747
ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.003737695274238994
RMSE: 0.0611366933538198
MAE: 0.016595873311879054
RMSLE: NaN
Mean Residual Deviance: 0.003737695274238994


In [ ]:
test_data_with_predicted_lat = test_data_with_last_known

lat_predictions_sin = gbm_lat_sin.predict(test_data_with_last_known)
test_data_with_last_known["latitude_sin"] = lat_predictions_sin
lat_predictions_cos = gbm_lat_cos.predict(test_data_with_last_known)
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

test_data_with_predicted_lat["latitude_sin"] = lat_predictions_sin
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

long_predictions_sin = gbm_long_sin.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_sin"] = long_predictions_sin
long_predictions_cos = gbm_long_cos.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_cos"] = long_predictions_cos

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [ ]:
# Convert sine and cosine values back to radians
lat_predictions_sin = lat_predictions_sin.as_data_frame()
lat_predictions_cos = lat_predictions_cos.as_data_frame()
long_predictions_sin = long_predictions_sin.as_data_frame()
long_predictions_cos = long_predictions_cos.as_data_frame()


lat_predictions_radians = np.arctan2(lat_predictions_sin, lat_predictions_cos)
long_predictions_radians = np.arctan2(long_predictions_sin, long_predictions_cos)

# Convert radians to degrees
lat_predictions_degrees = np.rad2deg(lat_predictions_radians)
long_predictions_degrees = np.rad2deg(long_predictions_radians)

# Print the first few rows to verify the conversion
print(lat_predictions_degrees.head())
print(long_predictions_degrees.head())

/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlering/Gruppe/.venv/lib/python3.12/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlering/Gruppe/.venv/lib/python3.12/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlering/Gruppe/.v

     predict
0  31.471013
1  15.716446
2  39.569700
3 -42.965954
4  48.537083
      predict
0  -80.769380
1  117.658928
2    9.321935
3  173.342732
4   -6.908912


In [ ]:
def create_prediction_visualization_data(validation_data):
    lat_val_sin = gbm_lat_sin.predict(validation_data)
    lat_val_cos = gbm_lat_cos.predict(validation_data)
    long_val_sin = gbm_long_sin.predict(validation_data)
    long_val_cos = gbm_long_cos.predict(validation_data)

    lat_val_sin = lat_val_sin.as_data_frame()
    lat_val_cos = lat_val_cos.as_data_frame()
    long_val_sin = long_val_sin.as_data_frame()
    long_val_cos = long_val_cos.as_data_frame()

    validation_data = validation_data.as_data_frame()

    lat_val_radians = np.arctan2(lat_val_sin, lat_val_cos)
    long_val_radians = np.arctan2(long_val_sin, long_val_cos)

    evaluation_lat_radians = np.arctan2(
        validation_data["latitude_sin"], validation_data["latitude_cos"]
    )
    evaluation_long_radians = np.arctan2(
        validation_data["longitude_sin"], validation_data["longitude_cos"]
    )

    # Convert radians to degrees
    lat_val_degrees = np.rad2deg(lat_val_radians)
    long_val_degrees = np.rad2deg(long_val_radians)

    evaluation_lat_degrees = np.rad2deg(evaluation_lat_radians)
    evaluation_long_degrees = np.rad2deg(evaluation_long_radians)

    eval_predictions = pd.concat([lat_val_degrees, long_val_degrees], axis=1)

    eval_actual = pd.concat([evaluation_lat_degrees, evaluation_long_degrees], axis=1)

    eval_predictions.columns = ["latitude_predicted", "longitude_predicted"]
    eval_actual.columns = ["latitude", "longitude"]

    eval = pd.DataFrame()
    eval[["latitude_predicted", "longitude_predicted"]] = eval_predictions
    eval[["latitude", "longitude"]] = eval_actual
    eval[["vesselId", "time"]] = validation_data[["vesselId", "time"]]
    eval.to_csv("eval_predictions.csv")


# create_prediction_visualization_data(validation_data_shifted)

In [ ]:
predictions = pd.concat([lat_predictions_degrees, long_predictions_degrees], axis=1)
predictions.columns = ["latitude_predicted", "longitude_predicted"]

In [ ]:
predictions["ID"] = test_data["ID"]
predictions = predictions[["ID", "longitude_predicted", "latitude_predicted"]]

In [ ]:
print(predictions.columns)

Index(['ID', 'longitude_predicted', 'latitude_predicted'], dtype='object')


In [ ]:
predictions.to_csv("predictions.csv", index=False)